In [ ]:
##### IMPORTACIÓN DE LIBRERÍAS #####
# Importación de pandas para manipulación de datos y tabula para leer archivos PDF.
import pandas as pd
import tabula

In [ ]:
# Definir la ruta al archivo PDF a procesar.
ruta = 'Extracto_Banco_Hipotecario.pdf'

##### LECTURA DEL ARCHIVO PDF Y EXTRACCIÓN DE DATOS #####
# Leer todas las páginas del PDF especificado usando Tabula.
# Se definen las áreas y columnas de interés para extraer las tablas de datos.
dfs = tabula.read_pdf(
    ruta, pages='all',
    relative_area=True, 
    relative_columns=True,
    area=[32, 0, 95, 100],
    columns=[11, 38, 42, 60, 75 , 85],
    pandas_options={'header': None},
    silent=True
)

In [ ]:
##### RENOMBRAR Y UNIR LOS DATAFRAMES #####
# Definir nombres para las columnas en el DataFrame.
columnas = ["Fecha", "Descripcion", "Suc", "Referencia", "Debitos", "Creditos", "Saldos"]

# Renombrar columnas de cada DataFrame en la lista para que tengan nombres consistentes.
dfs_renombrados = [df.rename(columns=dict(zip(df.columns, columnas))) for df in dfs]

# Concatenar todos los DataFrames en un único DataFrame `df_unido`.
df_unido = pd.concat(dfs_renombrados, ignore_index=True)

In [ ]:
##### FILTRADO Y SELECCIÓN DE FILAS #####
# Encontrar el último índice donde aparece "SALDO FINAL AL DIA" en la columna 'Descripcion'.
ultimo_indice = df_unido[df_unido['Descripcion'].str.contains('SALDO FINAL AL DIA', na=False)].index[-1]

# Crear un nuevo DataFrame `df_LastN` que incluye solo las filas hasta el último "SALDO FINAL AL DIA".
df_LastN = df_unido[0:ultimo_indice+1]

In [ ]:
##### LIMPIEZA Y COMPLETADO DE VALORES NULOS #####
# Eliminar las filas con valores NaN en la columna "Fecha" y crear una copia del DataFrame.
df_sin_nan = df_LastN.dropna(subset=['Fecha']).copy()

# Crear una columna de índice personalizado 'Indice' en el DataFrame `df_sin_nan`.
df_sin_nan = df_sin_nan.assign(Indice=range(1, len(df_sin_nan) + 1))

# Combinar `df_LastN` con `df_sin_nan` para incluir el índice en todas las filas originales, usando forward-fill para completar valores faltantes.
df_merge = df_LastN.merge(df_sin_nan[['Indice']], left_index=True, right_index=True, how='left')
df_merge['Indice'] = df_merge['Indice'].ffill()

# Reemplazar valores NaN en las columnas de texto 'Descripcion' y 'Referencia' con cadenas vacías.
df_merge['Descripcion'] = df_merge['Descripcion'].fillna('')
df_merge['Referencia'] = df_merge['Referencia'].fillna('')

In [ ]:
##### AGRUPACIÓN Y SUMA DE VALORES #####
# Agrupar datos por 'Indice' y combinar las descripciones, referencias, y sumar los valores numéricos.
df_agrupado = df_merge.groupby('Indice').agg({
    'Fecha': 'first',  # Tomar la primera fecha en cada grupo
    'Descripcion': lambda x: ' '.join(x),  # Concatenar descripciones
    'Referencia': lambda x: ' '.join(x),   # Concatenar referencias
    'Debitos': 'sum',  # Sumar valores en 'Debitos'
    'Creditos': 'sum',  # Sumar valores en 'Creditos'
    'Saldos': 'sum'     # Sumar valores en 'Saldos'
}).reset_index()

In [ ]:
##### FILTRADO DE DATOS #####
# Eliminar filas donde 'Descripcion' contiene "SALDO FINAL" o "DESCRIPCION" (ignorar mayúsculas).
df_filter = df_agrupado[~df_agrupado['Descripcion'].str.contains(r'SALDO FINAL|DESCRIPCION', case=False)].copy()

In [ ]:
##### CONVERSIÓN DE DATOS Y RELLENO DE VALORES NULOS #####
# Eliminar comas y convertir las columnas 'Debitos', 'Creditos' y 'Saldos' a tipo float.
df_filter['Debitos'] = df_filter['Debitos'].str.replace(',', '').astype(float)
df_filter['Creditos'] = df_filter['Creditos'].str.replace(',', '').astype(float)
df_filter['Saldos'] = df_filter['Saldos'].str.replace(',', '').astype(float)

# Rellenar valores NaN en las columnas 'Creditos', 'Debitos' y 'Saldos' con 0 y ajustar tipos.
df_filter['Creditos'] = df_filter['Creditos'].fillna(0).infer_objects(copy=False)
df_filter['Debitos'] = df_filter['Debitos'].fillna(0).infer_objects(copy=False)
df_filter['Saldos'] = df_filter['Saldos'].fillna(0).infer_objects(copy=False)

In [ ]:
##### CÁLCULO DE SALDO ACUMULADO #####
# Crear una columna 'Dif' que calcule la diferencia entre 'Creditos' y 'Debitos' más el saldo actual.
df_filter['Dif'] = df_filter['Creditos'] - df_filter['Debitos'] + df_filter['Saldos']

# Crear una columna 'Saldo Acum' para la suma acumulada de 'Dif'.
df_filter['Saldo Acum'] = df_filter['Dif'].cumsum().round(2)

# Eliminar las columnas innecesarias para el archivo final.
df_final = df_filter.drop(['Indice', 'Dif', 'Saldos'], axis=1)

In [ ]:
##### EXTRACCIÓN DE INFORMACIÓN ADICIONAL DEL PDF #####
# Extraer el periodo y CBU desde la primera página del PDF.
cbu_periodo = tabula.read_pdf(
    ruta, pages=1,
    relative_area=True, 
    relative_columns=True,
    area=[15, 0, 25, 100],
    columns=[58],
    pandas_options={'header': None},
    silent=True
)[0]

# Obtener las fechas de inicio y fin del periodo a partir del texto extraído.
periodo = cbu_periodo[1][0].split(':')[1].split('al')
desde = periodo[0].strip()
hasta = periodo[1].strip()

# Reformatear las fechas al formato YYYY-MM-DD.
fecha_desde = "-".join(reversed(desde.split("/")))
fecha_hasta = "-".join(reversed(hasta.split("/")))

# Extraer el CBU del texto.
cbu = cbu_periodo[0][1][-18:]

In [ ]:
##### EXPORTACIÓN DEL DATAFRAME FINAL A CSV #####
# Crear el nombre del archivo CSV basado en el número de cuenta (CBU) y el periodo.
nombre_excel = f"Nro_Cta_{cbu}_Periodo_{fecha_desde}_al_{fecha_hasta}.csv"

# Exportar el DataFrame final a un archivo CSV.
df_final.to_csv(nombre_excel, sep=';', encoding='latin-1', index=False)